In [1]:
# Torch
import torch
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler
# Data Handling
import numpy as np
import pandas as pd
# System
import os
import pprint
import itertools
# Custom
import models
import datasets
from helpers import utils

In [2]:
########################## FILE SYSTEM SETTINGS ###########################

savedir_base = "/home/jovyan/work/runs"
datadir = "/home/jovyan/work/DENMARK/250x250"

####################### EXPERIMENT SETTINGS ################################

experiment_id = "8bd70e101945e82afea8a5021b195720"
run_path = os.path.join(savedir_base, experiment_id)
exp_dict = utils.loadJSON(os.path.join(run_path, 'exp_dict.json'))

pprint.pprint(exp_dict)

{'batch_size': 1,
 'dataset': {'name': 'denmark', 'transform': 'rgb_normalize'},
 'dataset_size': {'train': 'all', 'val': 'all'},
 'lr': 1e-05,
 'max_epoch': 5,
 'model': {'base': 'fcn8_vgg16', 'name': 'cob'},
 'optimizer': 'adam'}


In [3]:
dataset_size = {'test': 'all'}
batch_size = 1

test_set = datasets.getDataset(dataset = exp_dict["dataset"]["name"],
                               split = "test",
                               datadir = datadir,
                               transform = exp_dict['dataset']['transform'],
                               dataset_size = dataset_size)

test_sampler = torch.utils.data.SequentialSampler(test_set)

test_loader = DataLoader(test_set,
                         sampler = test_sampler,
                         batch_size = batch_size,
                         num_workers = 2)

print("Testing Data loaded...")

Testing Data loaded...


In [4]:
n_classes = test_set.n_classes
model = models.getModel(exp_dict, n_classes).cuda()

run_model = utils.loadTorch(os.path.join(run_path, 'model_best.pth'))
model.loadStateDict(run_model)

print("Model loaded...")

Model loaded...


In [5]:
test_dict = model.testOnLoader(test_loader, savedir_images=os.path.join(run_path, "images"), n_images=30)

Testing. mIoU: 0.1677: 100%|██████████| 63/63 [00:04<00:00, 14.06it/s]


# EXPERIMENT REGION

In [ ]:
from models.losses import cob_loss
stop = True

with torch.no_grad():
    for i, batch in enumerate(test_loader):
        if stop:
            # Declare Eval Mode
            model.eval()

            # Load Data to GPU
            images = batch["images"].cuda()
            points = batch["points"].long().cuda()
            shapes = batch["shapes"]
            idx = batch["meta"]["index"]

            # Forward Prop
            logits = model.model_base.forward(images)

            # Sigmoid Function
            probs = logits.sigmoid().cpu().numpy()
            blobs = cob_loss.getBlobs(probs=probs)
            
            points_numpy = points.cpu().numpy()
            point_blobs = np.unique(blobs * points_numpy)
            
            if (np.unique(point_blobs) !=0 ).sum() > 1:
                blob_mask1 = blobs==point_blobs[1]
                blob_mask2 = blobs==point_blobs[2]
                
                model.visOnBatch(batch, "og.jpg")
                stop = False

In [ ]:
from PIL import Image, ImageDraw

for shape in shapes:
    flat_list = [item.item() for sublist in shape for item in sublist]
    img = Image.new('L', (250, 250), 0)
    ImageDraw.Draw(img).polygon(flat_list, outline=1, fill=1)
    shape_numpy = np.array(img)
    shape_mask = shape_numpy == 1
    
    blob_ids = np.unique(blobs * shape_mask)
    blob_ids = blob_ids[blob_ids != 0]
    
    print(blob_ids)
    
    running_iou = 0
    if len(blob_ids) == 0:
        iou_score = 0
    else:
        blob_masks = []
        for blob in blob_ids:
            blob_masks.append(blobs == blob)
            print((blobs == blob).sum())
        # convert into single mask and get intersection
        blob_union = np.logical_or.reduce(blob_masks)
        # get intersection adn union
        union = np.logical_or(blob_union, shape_mask).sum()
        intersection = np.logical_and(blob_union, shape_mask).sum()
        print(intersection)

        running_iou += intersection / union
    
    print(running_iou)
    

In [ ]:
from helpers import haven_viz
img_src = haven_viz.get_image(images.cpu(), denorm="rgb")
y_list, x_list = np.where(points_numpy.squeeze())

imagesl = []
for blob in point_blobs:
    if blob == 0:
        continue
    blob_mask = blobs==blob
    pred_img = haven_viz.mask_on_image(img_src, blob_mask.squeeze()))
    

img_labels = haven_viz.points_on_image(y_list, x_list, img_src)
img_pred = haven_viz.mask_on_image(img_src, blobs.squeeze())
img_pred_shape = haven_viz.mask_on_image(img_src, np.array(img).squeeze())

stitched_image = np.hstack([img_labels, img_pred, img_pred_shape])
stitched_image2 = np.hstack(imagesl)
haven_viz.save_image('test.jpg', stitched_image)
haven_viz.save_image('test2.jpg', stitched_image2)

In [ ]:
one = np.array([[0, 1, 0], [0, 1, 0]]) ==1
two = np.array([[0, 1, 0], [0, 1, 0]]) == 1
three = np.array([[1, 0, 1], [0, 1, 0]]) == 1

In [ ]:
np.logical_or.reduce([three, two])